# Chapter 8 exercises

In [1]:
import pandas as pd
import numpy as np

## 8.3

Consider three binary variables $a, b, c \in \{0, 1\}$ having the joint distribution given by the table below. Show by direct evaluation that this distribution has the property that $a$ and $b$s are marginally dependent, so that $p(a,b) \neq p(a)p(b)$, but that they become independent when conditioned on $c$ so that $p(a,b|c) = p(a|c)p(b|c)$ fo both $c=0$ and $c=1$

In [2]:
p = np.array([
    [0, 0, 0, 0.192],
    [0, 0, 1, 0.144],
    [0, 1, 0, 0.048],
    [0, 1, 1, 0.216],
    [1, 0, 0, 0.192],
    [1, 0, 1, 0.064],
    [1, 1, 0, 0.048],
    [1, 1, 1, 0.096],
])

p = pd.DataFrame(p, columns=["a", "b", "c", "p_abc"])
p = p.set_index(["a", "b", "c"])

#### Factorization without conditioning

In [25]:
# p(a, b)
p.xs(0, level=2) + p.xs(1, level=2)

p_abc
a   b         
0.0 0.0  0.336
    1.0  0.264
1.0 0.0  0.256
    1.0  0.144

In [26]:
# p(a)p(b)
p_marg = p.xs(0, level=2) + p.xs(1, level=2)
p_a = p_marg.xs(0, level=-1) + p_marg.xs(1, level=-1)
p_b = p_marg.xs(0, level=0) + p_marg.xs(1, level=0)

p_a["key"] = 1
p_b["key"] = 1

p_marg = pd.merge(p_a.reset_index(), p_b.reset_index(), on="key",
         suffixes=("_a", "_b")).drop("key", axis=1)

p_marg.set_index(["a", "b"]).prod(axis=1)

a    b  
0.0  0.0    0.3552
     1.0    0.2448
1.0  0.0    0.2368
     1.0    0.1632
dtype: float64

#### Factorization with conditioning

In [29]:
# p(a,b|c=0)
p_marg = p.xs(0, level=-1)
p_marg = p_marg / p_marg.sum()
p_marg

p_abc
a   b         
0.0 0.0    0.4
    1.0    0.1
1.0 0.0    0.4
    1.0    0.1

In [30]:
# p(a|c=0)p(b|c=0)
p_a = p_marg.sum(level=0)
p_b = p_marg.sum(level=1)

p_a = p_a.reset_index().assign(key=1)
p_b = p_b.reset_index().assign(key=1)

p_fact = (pd.merge(p_a, p_b, on="key", suffixes=("_a", "_b"))
            .drop("key", axis=1)
            .set_index(["a", "b"]))

p_fact.prod(axis=1)

a    b  
0.0  0.0    0.4
     1.0    0.1
1.0  0.0    0.4
     1.0    0.1
dtype: float64

## 8.4

Evaluate the distributions $p(a)$, $p(b | c)$, and $p(c | a)$ corresponding to the joint distribution given in Table 8.2. Hence show by direct evaluation that $p(a, b, c) = p(a)p(c | a)p(b | c)$. Draw the corresponding directed graph.

In [60]:
p_a = p.sum(level=0)

In [102]:
a = 1
p_a["key"] = [1,2]
p_a

,p_abc,key
a,,
0.0,0.6,1
1.0,0.4,2


In [108]:
# p(c|a)
p_c_giv_a = p.sum(level=["a", "c"]) / p.sum(level=["a", "c"]).sum(level=0)
p_c_giv_a["key"] = [1, 1, 2, 2]
p_c_giv_a

p_abc  key
a   c              
0.0 0.0    0.4    1
    1.0    0.6    1
1.0 0.0    0.6    2
    1.0    0.4    2

In [334]:
# p(b|c)
p_b_giv_c = p.sum(level=["b", "c"]) / p.sum(level=["b", "c"]).sum(level=1)
p_b_giv_c["key"] = [1, 2, 1, 2]
p_b_giv_c

p_abc  key
b   c              
0.0 0.0    0.8    1
    1.0    0.4    2
1.0 0.0    0.2    1
    1.0    0.6    2

In [335]:
fact = p_a.merge(p_c_giv_a.reset_index(), on="key")
fact = fact.drop(["key"], axis=1).set_index(["a", "c"])
fact = fact.prod(axis=1)
fact = pd.DataFrame(fact, columns=["p"])
fact["key"] = [1, 2, 1, 2]

fact = fact.reset_index().merge(p_b_giv_c.reset_index(), on="key")
fact = fact.rename({"c_x": "c"}, axis=1).drop(["key", "c_y"], axis=1)
fact = fact.groupby(["a", "c", "b"]).sum().prod(axis=1)

fact.name="p_factorized"

Final Answer:

Contains a bug: c-level is swapped

In [338]:
p.join(fact, on=["a", "b", "c"])

p_abc  p_factorized
a   b   c                       
0.0 0.0 0.0  0.192         0.192
        1.0  0.144         0.048
    1.0 0.0  0.048         0.144
        1.0  0.216         0.216
1.0 0.0 0.0  0.192         0.192
        1.0  0.064         0.048
    1.0 0.0  0.048         0.064
        1.0  0.096         0.096